# Export top of the book & trades using dedicated service - F#

### Overview
Retrieving trades and top of the book data points over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To highlight individual tradres among market activity. Please refer to <i><a href="">bestex-pointintime</a></i> and <i><a href="">bestex-interval</a></i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades and best Bid/Ask extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run Top of the book & Trades extraction sample

### Step 1: Install packages

In [ ]:
#r "nuget: Systemathics.Apis, 2.32.*"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open type Systemathics.Apis.Services.Tick.V1.TickTradesAndBookService
open type Systemathics.Apis.Services.Tick.V1.TickTradesAndBookRequest
open type Systemathics.Apis.Services.Tick.V1.TickTradesAndBookResponse
open type Systemathics.Apis.Type.Shared.V1.Trade
open type Systemathics.Apis.Type.Shared.V1.Book
open type Systemathics.Apis.Type.Shared.V1.BookUpdates

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData()

// Create communication channel
let channel = ChannelHelpers.GetChannel()

### Step 3: Create and process request
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
// Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
// The ICE ticker
let ticker = "E:BNP"
// The book limit : get the top 10 best limits
let maxDepth = 10
// The book update type: snapshot only or first snapshot then incremental
let updatesType = BookUpdates.Incrementals

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
let start = new DateTime(2021, 11, 01)
let end' = new DateTime(2021, 11, 01)
let dateIntervals = 
    new DateInterval(
        StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
        EndDate = new Date ( Year = end'.Year, Month = end'.Month, Day = end'.Day )
    )

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
    new TimeInterval(
        StartTime = new TimeOfDay ( Hours = 14, Minutes = 00, Seconds = 51 ),
        EndTime = new TimeOfDay ( Hours = 14, Minutes = 01, Seconds = 00)
    )

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

The following code snippet create the market data request and instantiate the service:

In [ ]:
// Generate the tick data request
let request = new TickTradesAndBookRequest ( Constraints = constraints, BookUpdates = updatesType, MaxDepth = maxDepth, Adjustment = false )
let identifier1 = new IdentifierAndLevel ( Provider = "ICE", Exchange = "EQUITY_L2_973", Ticker = ticker, Level = Level.TradesAndBook)
request.Identifiers.Add([| identifier1 |]) 
let identifier2 = new IdentifierAndLevel ( Provider = "ICE", Exchange = "787", Ticker = ticker, Level = Level.Trades)
request.Identifiers.Add([| identifier2 |])

// Instantiate the tick data service client
let service = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel)

### Step 4: Export data

#### 4.1 Retrieve top of the book & trades

We basically receive 2 types of data in the reponse
1. Trade data (price, size, trade id and trade condition)
2. Book data (a list of up to *maxDepth* limits on booth Bid and Ask sides of the book)

The following code exports to a file the time increasing trade (scalar) or book data (matrix)

In [ ]:
// Process tick data stream service: get price / size / timestamp
open System.Collections.Generic


// Used to process IAsyncEnumerable with F# along with a function f returning unit to process data
let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()

        }
    task.Wait()

// Function used to process data from stream
let displayResponse (t : StreamWriter) (item: TickTradesAndBookResponse)  =
    if not (isNull item.Mapping) then
        let mapping = item.Mapping
        mapping.Values
        |> Seq.iter(fun a ->  t.WriteLine($"# {a.Value} => {a.Identifier.Provider}/{a.Identifier.Exchange}/{a.Identifier.Ticker}"))
    else
        let ts = item.Data.TimeStamp.ToDateTime()

        if not (isNull item.Data.Book) then
            let book = item.Data.Book
            let header = $"{ts:O} [{item.Data.Mapping}] [BOOK]"
            t.WriteLine()
            t.WriteLine(header)
            // In snapshot mode, the book limits are sorted in the natural order
            // This code leverage the fact we know that bids/asks always come complete/ordered, it wouldn't work in incremental updates mode
            let maxDepth = Math.Max(book.Bid.Count, book.Ask.Count)
            for i = 0 to (maxDepth - 1) do 
                let bidPrice = if i < book.Bid.Count then $"{book.Bid[i].Price:N4}" else String.Empty
                let bidSize  = if i < book.Bid.Count then $"{book.Bid[i].Size:N0}"  else String.Empty
                let bidDepth = if i < book.Bid.Count then $"{book.Bid[i].Depth:N0}" else String.Empty
                let askPrice = if i < book.Ask.Count then $"{book.Ask[i].Price:N4}" else String.Empty
                let askSize  = if i < book.Ask.Count then $"{book.Ask[i].Size:N0}"  else String.Empty
                let askDepth = if i < book.Ask.Count then $"{book.Ask[i].Depth:N0}" else String.Empty
                let f = $"{bidDepth, 10} {bidSize, 10} {bidPrice, 10} {askPrice, 10} {askSize, 10} {askDepth, 10}"
                t.WriteLine(f)

        if not (isNull item.Data.Trade) then
            let trade = item.Data.Trade
            let header = $"{ts:O} [{item.Data.Mapping}] [TRADE]"
            t.WriteLine()
            t.WriteLine(header)
            t.WriteLine($"{trade.Size}@{trade.Price} Id={trade.Id} Condition={trade.Condition}")


let d = new DirectoryInfo("output")
if (not d.Exists) then d.Create()
let t = new StreamWriter(File.OpenWrite("output/mbl_incrementals.txt"))
let call = service.TickTradesAndBook(request, headers)
iterAsync (call.ResponseStream.ReadAllAsync()) (displayResponse t)
t.Close()